In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

priyanshumalaviya910_pigbehaviordetection_path = kagglehub.dataset_download('priyanshumalaviya910/pigbehaviordetection')
priyanshumalaviya910_models_path = kagglehub.dataset_download('priyanshumalaviya910/models')

print('Data source import complete.')


Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exist

In [ ]:
!pip install ultralytics opencv-python-headless munkres torch torchvision pandas scikit-learn tqdm

In [ ]:
import os
import time
import json
import glob
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from munkres import Munkres
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
DEVICE        = 'cuda' if torch.cuda.is_available() else 'cpu'
FPS           = 6
CONF_THRESH   = 0.25  # lower from 0.9 to 0.25
NMS_IOU       = 0.4
MAX_PIGS      = 8
TRACKER_LIFE  = 8      # frames before we drop a tracklet
MOVE_THRESH   = 2.5    # cm over 2s window
FEEDER_POS    = np.array([640,700])
WATER1_POS    = np.array([200,710])
WATER2_POS    = np.array([1080,710])

In [ ]:
ANNOTATED_ROOT   = '/kaggle/input/pigbehaviordetection/annotated'
RESULTS_ROOT     = '/kaggle/working/results_dataset'
OUT_ROOT         = '/kaggle/working/predictions'
RESNET_WEIGHTS   = '/kaggle/input/models/posture_resnet18.pth'
YOLO_WEIGHTS     = '/kaggle/input/models/best.pt'
ACCURACY_REPORT  = '/kaggle/working/accuracy.txt'

os.makedirs(OUT_ROOT, exist_ok=True)
os.makedirs(RESULTS_ROOT, exist_ok=True)

In [ ]:
DETECTION_MAP = {
    4:'eat', 5:'drink',
    102:'lying', 101:'standing',
    2:'moving', 3:'moving'
}
LABELS    = ['lying','standing']
LABEL_MAP = {l:i for i,l in enumerate(LABELS)}

In [ ]:
def regenerate_pen_mask(background_path):
    """Create binary pen mask from background.png via thresholding."""
    img = cv2.imread(background_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY_INV)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))
    return (mask>0).astype(np.uint8)

def pixel_to_cm(delta_px, depth_scale):
    return delta_px * depth_scale

def bbox_centroid(bbox):
    x,y,w,h = bbox
    return ((x+w/2),(y+h/2))

def compute_iou(a,b):
    x1,y1,w1,h1 = a; x2,y2,w2,h2 = b
    xi1, yi1 = max(x1,x2), max(y1,y2)
    xi2, yi2 = min(x1+w1,x2+w2), min(y1+h1,y2+h2)
    inter = max(0, xi2-xi1)*max(0, yi2-yi1)
    union = w1*h1 + w2*h2 - inter
    return inter/union if union>0 else 0

In [ ]:
yolo_det = YOLO(YOLO_WEIGHTS)

def detect_frame(frame, pen_mask):
    # run your custom model once per frame
    res = yolo_det.predict(
        source=[frame],
        imgsz=416,
        conf=CONF_THRESH,
        iou=NMS_IOU,
        device=DEVICE,
        verbose=False
    )[0]
    boxes = []
    H,W = frame.shape[:2]
    for box in res.boxes.xyxy.cpu().numpy():
        x1,y1,x2,y2 = box.astype(int)
        cx,cy = (x1+x2)//2, (y1+y2)//2
        if 0 <= cx < W and 0 <= cy < H and pen_mask[cy,cx]:
            boxes.append([x1,y1,x2-x1,y2-y1])
    return boxes[:MAX_PIGS]

In [ ]:
class Tracklet:
    def __init__(self, tid, bbox, frame, img):
        self.id = tid
        self.bboxes = [(frame, bbox)]
        self.centroids = [bbox_centroid(bbox)]
        self.tracker = cv2.legacy.TrackerMOSSE_create()
        self.tracker.init(img, tuple(bbox))
        self.last_frame = frame
        self.missed = 0
        self.behaviors = []

    def predict(self, frame, img):
        ok, box = self.tracker.update(img)
        if ok:
            self.bboxes.append((frame, list(box)))
            self.centroids.append(bbox_centroid(box))
            self.last_frame = frame
            self.missed = 0
            return True
        else:
            self.missed += 1
            return False

def assign_detections(tracklets, detections, frame, img):
    m = Munkres()
    cost = []
    for t in tracklets:
        cost.append([1 - compute_iou(t.bboxes[-1][1], d) for d in detections])
    assigned = set()
    if cost:
        pairs = m.compute(cost)
        for ti, di in pairs:
            if cost[ti][di] < 0.7:
                tracklets[ti].tracker = cv2.legacy.TrackerMOSSE_create()
                tracklets[ti].tracker.init(img, tuple(detections[di]))
                tracklets[ti].bboxes.append((frame, detections[di]))
                tracklets[ti].centroids.append(bbox_centroid(detections[di]))
                tracklets[ti].missed = 0
                assigned.add(di)
    return [dets[i] for i in range(len(dets)) if i not in assigned]

In [ ]:
posture_model = models.resnet18()
posture_model.fc = nn.Linear(posture_model.fc.in_features, 2)
posture_model.load_state_dict(torch.load(RESNET_WEIGHTS, map_location=DEVICE))
posture_model.to(DEVICE).eval()
posture_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

def classify_posture(crop):
    x = posture_tf(crop).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        out = posture_model(x).argmax(1).item()
    return LABELS[out]

In [ ]:
def classify_posture(crop):
    x = posture_tf(crop).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        out = posture_model(x).argmax(1).item()
    return LABELS[out]

In [ ]:
DETECT_EVERY = 1

In [ ]:
for date_dir in sorted(os.listdir(ANNOTATED_ROOT)):
    day_p = os.path.join(ANNOTATED_ROOT, date_dir)
    if not os.path.isdir(day_p):
        continue

    for clip in sorted(os.listdir(day_p)):
        clip_p = os.path.join(day_p, clip)
        cap    = cv2.VideoCapture(os.path.join(clip_p, 'color.mp4'))
        if not cap.isOpened():
            continue

        pen_mask    = regenerate_pen_mask(os.path.join(clip_p, 'background.png'))
        depth_scale = np.load(os.path.join(clip_p, 'depth_scale.npy'))

        tracklets, next_id = [], 0
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 1) DETECT & TRACK
        for f in tqdm(range(frame_count), desc=f"{date_dir}/{clip}"):
            ret, frame = cap.read()
            if not ret:
                break

            # a) detect only every N frames
            if f % DETECT_EVERY == 0:
                small = cv2.resize(frame, (640, 360))
                res   = yolo_det(small, imgsz=(640, 384), conf=CONF_THRESH, iou=NMS_IOU)[0]
                boxes = res.boxes.xyxy.cpu().numpy()
                sx, sy = frame.shape[1]/640, frame.shape[0]/360
                dets = []
                for x1, y1, x2, y2 in boxes:
                    cx = int(((x1+x2)/2)*sx)
                    cy = int(((y1+y2)/2)*sy)
                    if pen_mask[cy, cx]:
                        dets.append([
                            int(x1*sx), int(y1*sy),
                            int((x2-x1)*sx), int((y2-y1)*sy)
                        ])
            else:
                dets = []

            # b) update trackers & spawn new
            for t in tracklets:
                t.predict(f, frame)
            tracklets = [t for t in tracklets if t.missed < TRACKER_LIFE]
            new_bs = assign_detections(tracklets, dets, f, frame)
            for b in new_bs:
                tracklets.append(Tracklet(next_id, b, f, frame))
                next_id += 1

            # c) on‐the‐fly classification
            to_crop, owners = [], []
            for t in tracklets:
                if t.last_frame != f:
                    continue

                # moving?
                idx = len(t.centroids) - 1
                if idx >= 2*FPS:
                    dx_px = np.linalg.norm(
                        np.array(t.centroids[idx]) - np.array(t.centroids[idx-2*FPS])
                    )
                    if pixel_to_cm(dx_px, depth_scale) > MOVE_THRESH:
                        t.behaviors.append((f, 'moving', t.bboxes[-1][1]))
                        continue

                to_crop.append(t.bboxes[-1][1])
                owners.append(t)

            if to_crop:
                tensors = []
                for x, y, w, h in to_crop:
                    x1, y1 = max(0, int(x)), max(0, int(y))
                    x2 = min(frame.shape[1], int(x+w))
                    y2 = min(frame.shape[0], int(y+h))
                    crop = frame[y1:y2, x1:x2]
                    if crop.size == 0:
                        tensors.append(None)
                    else:
                        pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
                        tensors.append(posture_tf(pil))

                batch = torch.stack([t for t in tensors if t is not None]).to(DEVICE)
                with torch.no_grad():
                    preds = posture_model(batch).argmax(1).cpu().numpy()

                pi = 0
                for t, tensor in zip(owners, tensors):
                    if tensor is None:
                        beh = 'standing'
                    else:
                        # compute orientation
                        bbox = t.bboxes[-1][1]
                        x, y, w, h = map(int, bbox)
                        crop = frame[y:y+h, x:x+w]
                        if crop.size == 0:
                            beh = 'standing'
                        else:
                            gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
                            M    = cv2.moments(gray)
                            if M['mu20']+M['mu02'] > 0:
                                ang = 0.5*np.arctan2(2*M['mu11'], M['mu20']-M['mu02'])
                            else:
                                ang = 0.0
                            ori = np.array([np.cos(ang), np.sin(ang)], dtype=np.float32)
                            c0  = np.array(t.centroids[-1], dtype=np.float32)
                            def unit(v): return v/(np.linalg.norm(v)+1e-6)
                            if ori.dot(unit(FEEDER_POS - c0)) > 0.5:
                                beh = 'eat'
                            elif ori.dot(unit(WATER1_POS - c0)) > 0.5 or ori.dot(unit(WATER2_POS - c0)) > 0.5:
                                beh = 'drink'
                            else:
                                beh = LABELS[preds[pi]]
                            pi += 1
                    t.behaviors.append((f, beh, t.bboxes[-1][1]))

        cap.release()

        # 2) EXPORT JSON
        out = {
            "videoFileName":    "color.mp4",
            "fullVideoFilePath": f"{date_dir}/{clip}/",
            "stepSize":         1.0/FPS,
            "config": {
                "stepSize":          1.0/FPS,
                "playbackRate":      0.4,
                "imageMimeType":     "image/jpeg",
                "imageExtension":    ".jpg",
                "framesZipFilename": "extracted-frames.zip",
                "consoleLog":        False
            },
            "objects": []
        }
        for t in tracklets:
            frames, last = [], None
            for frm, beh, bb in t.behaviors:
                if beh != last:
                    frames.append({
                        "frameNumber": frm//3,
                        "bbox": {
                            "x":      int(bb[0]),
                            "y":      int(bb[1]),
                            "width":  int(bb[2]),
                            "height": int(bb[3])
                        },
                        "isGroundTruth": False,
                        "visible":       True,
                        "behaviour":     beh
                    })
                    last = beh
            out["objects"].append({"id": str(t.id), "frames": frames})

        dst = os.path.join(OUT_ROOT, date_dir, clip)
        os.makedirs(dst, exist_ok=True)
        with open(os.path.join(dst, 'output.json'), 'w') as f:
            json.dump(out, f, indent=2)


2019_11_05/000002:   0%|          | 0/1800 [00:00<?, ?it/s]


0: 224x384 8 pigs, 9.7ms
Speed: 1.4ms preprocess, 9.7ms inference, 1.6ms postprocess per image at shape (1, 3, 224, 384)


2019_11_05/000002:   0%|          | 1/1800 [00:00<03:22,  8.90it/s]


0: 224x384 8 pigs, 6.7ms
Speed: 1.4ms preprocess, 6.7ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 384)


2019_11_05/000002:   0%|          | 4/1800 [00:00<01:35, 18.81it/s]

...

2019_12_10/000078: 100%|█████████▉| 1793/1800 [01:43<00:00, 11.54it/s]


0: 224x384 2 pigs, 6.5ms
Speed: 1.0ms preprocess, 6.5ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 384)


2019_12_10/000078: 100%|█████████▉| 1797/1800 [01:43<00:00, 12.84it/s]


0: 224x384 2 pigs, 6.7ms
Speed: 1.0ms preprocess, 6.7ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 384)


2019_12_10/000078: 100%|██████████| 1800/1800 [01:43<00:00, 17.37it/s]


In [ ]:
total_tp=total_fp=total_fn=0
report_lines = ["Date,Clip,AP(%),TP(%),FP(%),Missed(%)"]
for date_dir in sorted(os.listdir(ANNOTATED_ROOT)):
    day_p = os.path.join(ANNOTATED_ROOT,date_dir)
    for clip in sorted(os.listdir(day_p)):
        gt_j   = os.path.join(day_p,clip,'output.json')
        pred_j = os.path.join(OUT_ROOT,date_dir,clip,'output.json')
        if not os.path.isfile(gt_j) or not os.path.isfile(pred_j): continue
        gt = json.load(open(gt_j))['objects']
        pr = json.load(open(pred_j))['objects']
        # flatten bbox lists
        gt_boxes = [ (o['id'],f['frameNumber'],tuple(f['bbox'].values()))
                     for o in gt for f in o['frames'] ]
        pr_boxes = [ (o['id'],f['frameNumber'],tuple(f['bbox'].values()))
                     for o in pr for f in o['frames'] ]
        matched = set()
        tp=fp=0
        for pid,fr,bb in pr_boxes:
            best_iou,best = 0,-1
            for i,(gpid,gfr,gbb) in enumerate(gt_boxes):
                if i in matched or gfr!=fr: continue
                iou = compute_iou(bb,gbb)
                if iou>best_iou:
                    best_iou,best = iou,i
            if best_iou>=0.5:
                tp +=1; matched.add(best)
            else:
                fp +=1
        fn = len(gt_boxes)-len(matched)
        total_tp+=tp; total_fp+=fp; total_fn+=fn
        ap = tp/(tp+fp)*100 if tp+fp>0 else 0
        tp_pct = tp/(tp+fn)*100 if tp+fn>0 else 0
        fp_pct = fp/(tp+fp)*100 if tp+fp>0 else 0
        fn_pct = fn/(tp+fn)*100 if tp+fn>0 else 0
        report_lines.append(f"{date_dir},{clip},{ap:.2f},{tp_pct:.2f},{fp_pct:.2f},{fn_pct:.2f}")

# overall
o_ap = total_tp/(total_tp+total_fp)*100 if total_tp+total_fp>0 else 0
o_tp = total_tp/(total_tp+total_fn)*100 if total_tp+total_fn>0 else 0
o_fp = total_fp/(total_tp+total_fp)*100 if total_tp+total_fp>0 else 0
o_fn = total_fn/(total_tp+total_fn)*100 if total_tp+total_fn>0 else 0
report_lines.append(f"Whole,Set,{o_ap:.2f},{o_tp:.2f},{o_fp:.2f},{o_fn:.2f}")

with open(ACCURACY_REPORT,'w') as f:
    f.write("\n".join(report_lines))

print("Pipeline complete. See accuracy report at", ACCURACY_REPORT)

Pipeline complete. See accuracy report at /kaggle/working/accuracy.txt
